<a href="https://colab.research.google.com/github/leohcar/P_IA025A_2022S1/blob/main/entrega_final/Proyecto_encoder_resnet18_smalldataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Encoder

In [1]:
nome = 'Carlos Leonardo Ancasi Hinostroza'

print(f'Meu nome é {nome}')

Meu nome é Carlos Leonardo Ancasi Hinostroza


# Importação das bibliotecas

In [2]:
import collections
import itertools
import functools
import math
import random

import torch
import torchvision
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook

from torchvision.datasets import MNIST

In [3]:
# Check which GPU we are using
!nvidia-smi

Sat Jul 16 01:22:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
print('GPU available:', device)

GPU available: cuda


In [5]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)

#Dataset e dataloader

In [18]:
!mkdir ./data

# transform = torchvision.transforms.Compose(
#     [torchvision.transforms.Resize([224,224]),
#      torchvision.transforms.ToTensor(),
#      torchvision.transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
transform = torchvision.transforms.Compose(
     [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                             download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                            download=True, transform=transform)

mkdir: cannot create directory ‘./data’: File exists
Files already downloaded and verified
Files already downloaded and verified


In [19]:
print(f'numero de imagenes {len(train_dataset)}')
img, target = train_dataset[0]
print(img.size())

numero de imagenes 50000
torch.Size([3, 32, 32])


In [20]:
train_size = 10
val_size = 10000
rest_size = 50000 - train_size - val_size

train_dataset, val_dataset, _ = torch.utils.data.random_split(train_dataset, [train_size, val_size,rest_size])

# train_size = 40000
# val_size = 10000

# train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

In [21]:
batch_size = 50

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print('Número de minibatches de trenamento:', len(train_dataloader))
print('Número de minibatches de validação:', len(val_dataloader))
print('Número de minibatches de teste:', len(test_dataloader))

x_train, y_train = next(iter(train_dataloader))
print("\nDimensões dos dados de um minibatch:", x_train.size())
print("Valores mínimo e máximo dos pixels: ", torch.min(x_train), torch.max(x_train))
print("Tipo dos dados das imagens:         ", type(x_train))
print("Tipo das classes das imagens:       ", type(y_train))

Número de minibatches de trenamento: 1
Número de minibatches de validação: 200
Número de minibatches de teste: 200

Dimensões dos dados de um minibatch: torch.Size([10, 3, 32, 32])
Valores mínimo e máximo dos pixels:  tensor(-1.) tensor(1.)
Tipo dos dados das imagens:          <class 'torch.Tensor'>
Tipo das classes das imagens:        <class 'torch.Tensor'>


#Bloco Básico da Resnet

In [22]:
from collections import OrderedDict
from torch import nn
from torch import Tensor


class MyBlock(nn.Module):

    def __init__(
        self,
        inplanes: int,
        planes: int,
        identity_downsample=None,
        stride: int = 1,
    ) -> None:
        super(MyBlock, self).__init__()
        # Escreva seu código aqui.
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=3, stride=stride, bias=False, padding=1)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, bias=False, padding=1)
        self.bn2 = nn.BatchNorm2d(planes)

        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        # Escreva seu código aqui.
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.identity_downsample is not None:
            identity = self.identity_downsample(x)
        out += identity
        out = self.relu(out)
        return out

In [23]:
class CNN_resnet(nn.Module):
    def __init__(self, image_channels, num_classes):

        super(CNN_resnet,self).__init__()

        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        #resnet layers
        self.layer1 = self.__make_layer(64, 64, stride=1)
        self.layer2 = self.__make_layer(64, 128, stride=2)
        self.layer3 = self.__make_layer(128, 256, stride=2)
        self.layer4 = self.__make_layer(256, 512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))

        self.fc = nn.Linear(512, num_classes)
      
    def __make_layer(self, in_channels, out_channels, stride):

        identity_downsample = None
        
        if stride != 1:
            identity_downsample = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, padding=0,bias=False),
                nn.BatchNorm2d(out_channels))
        return nn.Sequential(MyBlock(in_channels, out_channels, identity_downsample=identity_downsample, stride=stride),
            MyBlock(out_channels, out_channels),
            nn.Dropout(0.3))
    
    def forward(self, x):

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = self.avgpool(out)
        out = out.view(x.shape[0], -1)
        out = self.fc(out)
        return out


In [24]:
model = CNN_resnet(3,100).to(device)

In [25]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 11227876


In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
model_save_name = 'model_resnet18_smalldataset.pt'
path_modelo = F"/content/drive/My Drive/modelo_proyecto/entrega_03/pre_trained/{model_save_name}"

In [29]:
# Iniciar o treinamento
from tqdm import tqdm

n_epochs = 60
lr = 3.3e-4

img_channels = 3
num_classes = 100

model = CNN_resnet(img_channels,num_classes).to(device)

# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

checkpoint = {
    'img_channels': img_channels,
    'num_classes': num_classes,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict' : optimizer.state_dict(),
    'epochs': [],
    'loss_train': [],
    'loss_val': [],
    'accuracy_history': [],
    }
torch.save(checkpoint, path_modelo)

def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids)
    loss.backward()
    optimizer.step()

    return loss.item()

def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids)

    _, y_pred = torch.max(logits,1)
    accuracy_item = (y_pred.cpu().numpy() == target_ids.cpu().numpy()).mean()

    return accuracy_item,loss.item()

epochs = []
loss_history = []
loss_epoch_end = []
accuracy_history = []
total_trained_samples = 0

with torch.no_grad():
    loss_accuracy = [list(validation_step(val_input_ids.to(device), val_target_ids.to(device)))
        for val_input_ids, val_target_ids in val_dataloader]
    
    mean_accuracy = np.array(loss_accuracy)[:,0].mean()
    mean_loss = np.array(loss_accuracy)[:,1].mean()
    print(f'Accuracy: {mean_accuracy} Loss_Val: {mean_loss}')
    accuracy_history.append(mean_accuracy.item())

    checkpoint['accuracy_history'].append(mean_accuracy.item())
    checkpoint['loss_val'].append(mean_loss.item())
    torch.save(checkpoint, path_modelo)
    
for i in range(n_epochs):
    for x_train, y_train in tqdm(train_dataloader, total=len(train_dataloader), desc='Training'):
        loss = train_step(x_train.to(device), y_train.to(device))
        
        total_trained_samples += x_train.size(0)
        epochs.append(total_trained_samples / len(train_dataset))
        loss_history.append(loss)
    
    loss_epoch_end.append(loss)
    print(f'Epoch: {i:d}/{n_epochs - 1:d} Loss_Train: {loss}')
    checkpoint['epochs'].append(i)
    checkpoint['loss_train'].append(loss)
    
    with torch.no_grad():
        loss_accuracy = [validation_step(val_input_ids.to(device), val_target_ids.to(device))
            for val_input_ids, val_target_ids in val_dataloader]
        
        mean_accuracy = np.array(loss_accuracy)[:,0].mean()
        mean_loss = np.array(loss_accuracy)[:,1].mean()
        print(f'Accuracy: {mean_accuracy} Loss_Val: {mean_loss}')
        accuracy_history.append(mean_accuracy.item())

        checkpoint['accuracy_history'].append(mean_accuracy.item())
        checkpoint['loss_val'].append(mean_loss.item())

    torch.save(checkpoint, path_modelo)


Accuracy: 0.0103 Loss_Val: 4.605971441268921


Training: 100%|██████████| 1/1 [00:00<00:00, 30.12it/s]

Epoch: 0/59 Loss_Train: 5.143030643463135


Accuracy: 0.0066 Loss_Val: 4.605928225517273


Training: 100%|██████████| 1/1 [00:00<00:00, 35.84it/s]

...

Epoch: 58/59 Loss_Train: 0.004036168567836285


Accuracy: 0.0116 Loss_Val: 7.134869148731232


Training: 100%|██████████| 1/1 [00:00<00:00, 22.85it/s]

Epoch: 59/59 Loss_Train: 0.004276941064745188


Accuracy: 0.0116 Loss_Val: 7.1707777452468875


In [30]:
# continuar com o  treino
from tqdm import tqdm

n_epochs = 100
lr = 1e-4

checkpoint = torch.load(path_modelo)

model = CNN_resnet(checkpoint['img_channels'],checkpoint['num_classes']).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids)
    loss.backward()
    optimizer.step()

    return loss.item()

def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids)

    _, y_pred = torch.max(logits,1)
    accuracy_item = (y_pred.cpu().numpy() == target_ids.cpu().numpy()).mean()

    return accuracy_item,loss.item()

epochs = []
loss_history = []
loss_epoch_end = checkpoint['loss_train']
accuracy_history = checkpoint['accuracy_history']
total_trained_samples = 0
epochs_init = checkpoint['epochs'][-1]+1

for s in zip(checkpoint['accuracy_history'],checkpoint['loss_val'],checkpoint['epochs'],checkpoint['loss_train']):
    print(f'Accuracy: {s[0]} Loss_Val: {s[1]}')
    print(f'')
    print(f'Epoch: {s[2]:d}/{n_epochs - 1:d} Loss_Train: {s[3]}')
print(f"Accuracy: { checkpoint['accuracy_history'][-1] } Loss_Val: {checkpoint['loss_val'][-1]}")

    
for i in range(epochs_init,n_epochs):
    for x_train, y_train in tqdm(train_dataloader, total=len(train_dataloader), desc='Training'):
        loss = train_step(x_train.to(device), y_train.to(device))
        
        total_trained_samples += x_train.size(0)
        epochs.append(total_trained_samples / len(train_dataset))
        loss_history.append(loss)
    
    loss_epoch_end.append(loss)
    print(f'Epoch: {i:d}/{n_epochs - 1:d} Loss_Train: {loss}')
    checkpoint['epochs'].append(i)
    checkpoint['loss_train'].append(loss)
    
    with torch.no_grad():
        loss_accuracy = [validation_step(val_input_ids.to(device), val_target_ids.to(device))
            for val_input_ids, val_target_ids in val_dataloader]
        
        mean_accuracy = np.array(loss_accuracy)[:,0].mean()
        mean_loss = np.array(loss_accuracy)[:,1].mean()
        print(f'Accuracy: {mean_accuracy} Loss_Val: {mean_loss}')
        accuracy_history.append(mean_accuracy.item())

        checkpoint['accuracy_history'].append(mean_accuracy.item())
        checkpoint['loss_val'].append(mean_loss.item())

    torch.save(checkpoint, path_modelo)


Accuracy: 0.0103 Loss_Val: 4.605971441268921

Epoch: 0/99 Loss_Train: 5.143030643463135
Accuracy: 0.0066 Loss_Val: 4.605928225517273

Epoch: 1/99 Loss_Train: 4.736281394958496
Accuracy: 0.0094 Loss_Val: 4.606434383392334

Epoch: 2/99 Loss_Train: 5.011910438537598
Accuracy: 0.0121 Loss_Val: 4.607201008796692

Epoch: 3/99 Loss_Train: 4.317760944366455
Accuracy: 0.0092 Loss_Val: 4.608118152618408

Epoch: 4/99 Loss_Train: 3.363018035888672
Accuracy: 0.0088 Loss_Val: 4.609453721046448

Epoch: 5/99 Loss_Train: 2.526933431625366
Accuracy: 0.0087 Loss_Val: 4.61140881061554

Epoch: 6/99 Loss_Train: 2.132131338119507
Accuracy: 0.009000000000000001 Loss_Val: 4.613646538257599

Epoch: 7/99 Loss_Train: 1.8440231084823608
Accuracy: 0.0088 Loss_Val: 4.616847031116485

Epoch: 8/99 Loss_Train: 1.7438304424285889
Accuracy: 0.0088 Loss_Val: 4.620103540420533

Epoch: 9/99 Loss_Train: 0.8995041847229004
Accuracy: 0.0088 Loss_Val: 4.624507565498352

Epoch: 10/99 Loss_Train: 0.527597188949585
Accuracy: 0.008

Training: 100%|██████████| 1/1 [00:00<00:00,  8.84it/s]

Epoch: 60/99 Loss_Train: 0.0037779677659273148


Accuracy: 0.011899999999999999 Loss_Val: 6.865634505748749


Training: 100%|██████████| 1/1 [00:00<00:00, 37.43it/s]

...

Epoch: 98/99 Loss_Train: 5.330801286618225e-05


Accuracy: 0.0108 Loss_Val: 9.4773540019989


Training: 100%|██████████| 1/1 [00:00<00:00, 24.42it/s]

Epoch: 99/99 Loss_Train: 2.7870002668350935e-05


Accuracy: 0.010700000000000001 Loss_Val: 9.47459909439087


In [30]:
test_loader = DataLoader(test_dataset, batch_size=batch_size)

with torch.no_grad():
    loss_accuracy = [validation_step(val_input_ids.to(device), val_target_ids.to(device))
        for val_input_ids, val_target_ids in val_dataloader]
        
    mean_accuracy = np.array(loss_accuracy)[:,0].mean()
    mean_loss = np.array(loss_accuracy)[:,1].mean()

print(f'TEST: Accuracy: {mean_accuracy} Loss_Val: {mean_loss}')

TEST: Accuracy: 0.4742 Loss_Val: 2.729253958463669
